In [2]:
%reset -f
import sys
sys.path.append("/home/jose/Documents/Maestria/Codigo/OTI/build/")
import Calc_Cable as cab
import pyoti.sparse as oti
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import pyoti.core as coti

In [7]:
#Lectura de datos de entrada
l,h,L0,A0,E,w,cargas = cab.leer_entradas("Irvine_1")

#Pertubar parametros (no sirve para perturbar cargas y ubicacion s de las cargas)
order = 1
nbases = 6
e1 = oti.e(1, order=order)
e2 = oti.e(2, order=order)
e3 = oti.e(3, order=order)
e4 = oti.e(4, order=order)
e5 = oti.e(5, order=order)
e6 = oti.e(6, order=order)
#l = l + e1
#h = h + e2
L0 = L0 + e3
#A0 = A0 + e4
#E = E + e5
#w = w + e6

#convertir adimensionales
gamma,delta,beta,Psi,Sigmas = cab.param_adim(l,h,L0,A0,E,w,cargas)

#calcular las incognitas gi fi
gi_0, fi_0 = 0.3, 0.4 #valores iniciales para iterar en newton
gi,fi = cab.calc_gi_fi(order,nbases,gamma,delta,beta,Psi,Sigmas,gi_0,fi_0)

#Reacciones

X = gi*w*L0
Y0 = fi*w*L0

#dX/dL0 , dY0/dL0
d = 3
print("gi, fi")
print(gi.get_deriv(0), fi.get_deriv(0))
print(gi.get_deriv(d), fi.get_deriv(d))

#dX/dL0 , dY0/dL0
d = 3
print("X, Y0")
print(X.get_deriv(0), Y0.get_deriv(0))
print(X.get_im(d), Y0.get_im(d))

#puntos de la carga 1
n = 1
xi, eta, tau = cab.solu(gi,fi,beta,Psi,Sigmas,Sigmas[n],n)
x = xi*L0
y = eta*L0

#dx/dL0 , dy/dL0
d = 3
print("x, y")
print(x.get_deriv(0), y.get_deriv(0))
print(x.get_deriv(d), y.get_deriv(d))

gi, fi
[[0.35120816]] [[0.66022779]]
[[-0.95601517]] [[-0.03647264]]
X, Y0
[[19.0237752]] [[35.76233865]]
[[-37.15048198]] [[25.53388985]]
x, y
[[0.18283793]] [[0.27398744]]
[[-0.35738491]] [[0.24169833]]


In [46]:
def puntos(x0, n, ht=0):
    l,h,L0,A0,E,w,cargas = cab.leer_entradas("Irvine_1")
    L0 += ht
    gamma,delta,beta,Psi_oti,Sigmas_oti = cab.param_adim(l,h,L0,A0,E,w,cargas)
    Psi = Psi_oti.get_deriv(0)
    Sigmas = Sigmas_oti.get_deriv(0)
    gf = fsolve(func, x0, args=(gamma,delta,beta,Psi,Sigmas))
    s = Sigmas[n]
    sum_xi = 0
    sum_eta = 0
    for i in range(n):
        sum_xi = sum_xi + np.arcsinh((gf[1]-Psi[i+2]-Sigmas[i+1])/gf[0]) - np.arcsinh((gf[1]-Psi[i+1]-Sigmas[i+1])/gf[0])
        sum_eta = sum_eta + beta*Psi[i+2]*(Sigmas[i+1]-s) + np.sqrt(gf[0]**2 + (gf[1]-Psi[i+2]-Sigmas[i+1])**2) - np.sqrt(gf[0]**2 + (gf[1]-Psi[i+1]-Sigmas[i+1])**2)
    xi = gf[0]*(beta*s + np.arcsinh(gf[1]/gf[0]) - np.arcsinh((gf[1]-np.sum(Psi[:n+2])-s)/gf[0]) + sum_xi)
    eta = beta*s*(gf[1]-s/2) + np.sqrt(gf[0]**2 + gf[1]**2) - np.sqrt(gf[0]**2 + (gf[1]-np.sum(Psi[:n+2])-s)**2) + sum_eta
    p = np.array([xi,eta])*L0  
    return(p)

In [47]:
def root(x0, ht=0):
    l,h,L0,A0,E,w,cargas = cab.leer_entradas("Irvine_1")
    L0 += ht
    gamma,delta,beta,Psi_oti,Sigmas_oti = cab.param_adim(l,h,L0,A0,E,w,cargas)
    Psi = Psi_oti.get_deriv(0)
    Sigmas = Sigmas_oti.get_deriv(0)
    return fsolve(func, x0, args=(gamma,delta,beta,Psi,Sigmas))*w*L0

In [48]:
def func(x,gamma,delta,beta,Psi,Sigmas):
    f = np.zeros(2)
    sum_gi = 0
    sum_fi = 0
    for i in range(len(Sigmas)):
        sum_gi = sum_gi + np.arcsinh((x[1]-Psi[i+1]-Sigmas[i])/x[0]) - np.arcsinh((x[1]-Psi[i]-Sigmas[i])/x[0])
        sum_fi = sum_fi + beta*Psi[i+1]*(Sigmas[i]-1) + np.sqrt((x[1]-Psi[i+1]-Sigmas[i])**2 + x[0]**2) - np.sqrt((x[1]-Psi[i]-Sigmas[i])**2 + x[0]**2)
    f[0] = beta - gamma/x[0] + np.arcsinh(x[1]/x[0]) - np.arcsinh((x[1]-np.sum(Psi)-1)/x[0]) + sum_gi
    f[1] = beta*(x[1]-0.5) - delta + np.sqrt(x[1]**2+x[0]**2) - np.sqrt((x[1]-np.sum(Psi)-1)**2 + x[0]**2) + sum_fi
    return f

In [49]:
x0 = np.array([0.3, 0.4])
ht = 1e-10

In [50]:
XY0 = root(x0)
dXY0 = (root(x0,ht) - root(x0))/ht
print("(X, Y0) = ", XY0)
print("(dX/dL0, dY0/dL0) = ", dXY0)

n = 1
p = puntos(x0, n)
dp_L0 = (puntos(x0,n,ht) - puntos(x0,n))/ht
print("(x, y) = ", p)
print("(dx/dL0, dy/dL0) = ", dp_L0)

(X, Y0) =  [24.03646052 33.10896976]
(dX/dL0, dY0/dL0) =  [-67.85512596  27.51598061]
(x, y) =  [[0.2238339 ]
 [0.24128018]]
(dx/dL0, dy/dL0) =  [[-0.46187804]
 [ 0.43181153]]
